In [ ]:
from requests.auth import HTTPBasicAuth
from impala.dbapi import connect
from datetime import datetime
from pyhive import hive
import pandas as pd
import requests
import sqlparse
import json
import sys
import os
import re

: 

In [ ]:
'''Variables'''
## Atlas dev
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}
credentials = HTTPBasicAuth('atlasadmin', 'xNB8lZ!Dut')
atlas_urls = ["172.30.213.141","172.30.213.142"]
atlas_port = 31000

## Hive dev
hive_host = 'edh-master-01d.root.corp'
hive_port = 10000
hive_user = 'admin'


## Impala dev
impala_host = '172.30.213.211'
impala_port = 21050
impala_user = 'admin'

: 

In [3]:
#HIVE
def describe_table( hive_database, table_name ):
    #Creo la conexión y obtengo el cursos
    hive_conn = hive.connect(host=hive_host, port=hive_port, username=hive_user, database=hive_database)
    hive_cursor = hive_conn.cursor()

    # Obtengo el create
    hive_cursor.execute(f'DESCRIBE {hive_database}.{table_name}')
    databases = hive_cursor.fetchall()
    
    return databases

In [4]:
path_metadata = 'data_procesada/metadata_hive_completo.csv'
path_excel = 'data_procesada/mejora_campos_raw.csv'
df_metadata = pd.read_csv(path_metadata, sep='|')
df_mejoras = pd.read_csv(path_excel)

In [5]:
df_metadata['base_tabla'] = df_metadata['base'] +'.'+df_metadata['tabla']

In [6]:
df_metadata[df_metadata['base_tabla']=='de_ber_2cur.clientes_bsado']

,campo,tipo_dato,comentario,base,tabla,entidad,zona,base_tabla
20276,id_interno_gdtime,varchar(20),Dato Control Interno,de_ber_2cur,clientes_bsado,ber,cur,de_ber_2cur.clientes_bsado
20277,id_cliente_core,varchar(13),Código Cliente,de_ber_2cur,clientes_bsado,ber,cur,de_ber_2cur.clientes_bsado
20278,tipo_doc,varchar(2),Tipo Documento,de_ber_2cur,clientes_bsado,ber,cur,de_ber_2cur.clientes_bsado
20279,nro_doc,varchar(11),Numero Documento,de_ber_2cur,clientes_bsado,ber,cur,de_ber_2cur.clientes_bsado
20280,cod_pais,varchar(2),Código de País,de_ber_2cur,clientes_bsado,ber,cur,de_ber_2cur.clientes_bsado
20281,fecha_alta,varchar(6),Fecha de Alta,de_ber_2cur,clientes_bsado,ber,cur,de_ber_2cur.clientes_bsado
20282,fecha_modificacion,varchar(6),Fecha de Modificación,de_ber_2cur,clientes_bsado,ber,cur,de_ber_2cur.clientes_bsado
20283,fecha_vto_doc,varchar(6),Fecha de Vencimiento de Documento,de_ber_2cur,clientes_bsado,ber,cur,de_ber_2cur.clientes_bsado
20284,usuario_alta,varchar(8),Código Usuario de Alta,de_ber_2cur,clientes_bsado,ber,cur,de_ber_2cur.clientes_bsado
20285,usuario_modificacion,varchar(8),Código Usuario de Modificación,de_ber_2cur,clientes_bsado,ber,cur,de_ber_2cur.clientes_bsado


In [7]:
df_metadata[df_metadata['base_tabla']=='de_ber_1raw.sfb_bsado']

,campo,tipo_dato,comentario,base,tabla,entidad,zona,base_tabla
20319,glb_dtime,string,Dato Control Interno,de_ber_1raw,sfb_bsado,ber,raw,de_ber_1raw.sfb_bsado
20320,sco_ident,string,Código Cliente,de_ber_1raw,sfb_bsado,ber,raw,de_ber_1raw.sfb_bsado
20321,scotipdoc,string,Tipo Documento,de_ber_1raw,sfb_bsado,ber,raw,de_ber_1raw.sfb_bsado
20322,snu_docum,string,Numero Documento,de_ber_1raw,sfb_bsado,ber,raw,de_ber_1raw.sfb_bsado
20323,sco_pais,string,Código de País,de_ber_1raw,sfb_bsado,ber,raw,de_ber_1raw.sfb_bsado
20324,sfeusuadi,string,Fecha de Alta,de_ber_1raw,sfb_bsado,ber,raw,de_ber_1raw.sfb_bsado
20325,sfeusumod,string,Fecha de Modificación,de_ber_1raw,sfb_bsado,ber,raw,de_ber_1raw.sfb_bsado
20326,sfevtodoc,string,Fecha de Vencimiento de Documento,de_ber_1raw,sfb_bsado,ber,raw,de_ber_1raw.sfb_bsado
20327,snousuadi,string,Código Usuario de Alta,de_ber_1raw,sfb_bsado,ber,raw,de_ber_1raw.sfb_bsado
20328,snousumod,string,Código Usuario de Modificación,de_ber_1raw,sfb_bsado,ber,raw,de_ber_1raw.sfb_bsado


In [8]:
with open('data/json_completo.json') as f:
    file = f.read()
    json_file = json.loads(file)

#### Obtengo el diccionario que me permite saber cuál es la tabla en curado

In [9]:
pares_raw_cur = {}

for table_raw in json_file.keys():
    if '1raw' in table_raw:
        db_raw, table_name_raw = table_raw.split('.')
        # Obtenemos lista de tablas afectadas
        list_affected_tables = json_file[table_raw]['affected_tables']
        # Si afecta más de una tabla, debemos obtener solo la que posea la misma fuente en el nombre en curado
        if len(list_affected_tables)>1:
            source = table_name_raw.split('_')[-1]
            # Filtra la tabla que contiene la fuente
            cur_table = [t for t in list_affected_tables if source in t]
            cur_table = cur_table[0]
        if len(list_affected_tables)>0:
            # Si tiene 1 solo elemento, entonces esa es la tabla en curado
            cur_table = list_affected_tables[0]
        
        # Si llgué acá es porque tengo la tabla en raw + cur
        db_cur, table_name_cur = cur_table.split('.')

        create_raw = describe_table(db_raw, table_name_raw) 
        create_cur = describe_table(db_cur, table_name_cur) 
        if len(create_raw) == len(create_cur):
            #La cantidad de campos es igual
            # Unimos tablas raw + cur
            pares_raw_cur[table_raw] = cur_table

#### En base al diccionario y al df con nuevos comentarios actualizamos el df de metadata

In [10]:
for index, row in df_mejoras.iterrows():
    # Nombre de tabla a reemplazar en metadata
    nombre_tabla = row['Base.Tabla']

    # Obtengo el comentario nuevo
    create_table_nuevo = row['Create Table Nuevo']

    try:
        comentario_nuevo = create_table_nuevo.split('COMMENT')[-1].replace(';', '')
    except (AttributeError, IndexError) as e:
        # Existen filas vacías o el índice no existe
        continue

    # Limpio el resultado
    match = re.search(r"'(.*?)'", comentario_nuevo)
    
    if match:
        comentario_tabla = match.group(1)

        # Reemplazamos en RAW por el nuevo comentario
        cond_raw = (df_metadata['base_tabla'] == nombre_tabla) & (df_metadata['campo'].isna())
        df_metadata.loc[cond_raw, 'comentario'] = comentario_tabla

        #Obtengo la misma tabla en curado
        try:
            tabla_curado = pares_raw_cur[nombre_tabla]
            cond_cur = (df_metadata['base_tabla'] == tabla_curado) & (df_metadata['campo'].isna())
            df_metadata.loc[cond_cur, 'comentario'] = comentario_tabla
        except KeyError as e:
            continue

#### Eliminamos la columna 'base_tabla'

In [11]:
df_metadata = df_metadata.drop(columns=['base_tabla'])

In [12]:
df_metadata.to_csv('data_procesada/metadata_hive_comentarios_nuevos.csv', sep='|', index=False)